# <div align=center><b>Composing reader to create custom SeqData objects</b></div>
The previous two tutorials focused on generating SeqData objects that would commonly be used in genomics ML model training. The functions we used (`read_table`, `read_bigwig` and `read_bam`) are actual custom examples of using a more basic set of building blocks for reading genomic data. In this notebook, we dive deeper into these building blocks to illustrate how they can be used to create more custom SeqData objects.

In [1]:
import seqdata as sd
from pathlib import Path
sd.__version__

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


/cellar/users/aklie/opt/miniconda3/envs/eugene_tools/lib/python3.11/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
/cellar/users/aklie/opt/miniconda3/envs/eugene_tools/lib/python3.11/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
/cellar/users/aklie/opt/miniconda3/envs/eugene_tools/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as noteb

'0.1.2'

# SeqData readers

SeqData distinguishes between two types of readers: 1) `FlatReader` and 2) `RegionReader`. 

For the first major release of SeqData, there are a total of 5 readers available:
1. `Table` (FlatReader)
2. `BigWig` (RegionReader)
3. `BAM` (RegionReader)
4. `GenomeFasta` (RegionReader)
5. `FlatFile` (FlatReader)
6. `VCF` (RegionReader)

Each reader comes with a function that helps. Using `FlatFasta` as an example since we haven't used it yet:

In [2]:
from seqdata import read_flat_fasta

In [3]:
# Get file name
fname = Path(sd.__file__).resolve().parent.parent / 'tests' / 'data' / 'tangermeme.fa'
fname

PosixPath('/cellar/users/aklie/projects/ML4GLand/SeqData/tests/data/tangermeme.fa')

In [4]:
read_flat_fasta?

Signature:
read_flat_fasta(
    name: str,
    out: Union[str, pathlib.Path],
    fasta: Union[str, pathlib.Path],
    batch_size: int,
    fixed_length: bool,
    n_threads=1,
    overwrite=False,
) -> 'xr.Dataset'
Docstring:
Reads sequences from a "flat" FASTA file into xarray.

We differentiate between "flat" and "genome" FASTA files. A flat FASTA file is one
where each contig in the FASTA file is a sequence in our dataset. A genome FASTA file
is one where we may pull out multiple subsequences from a given contig.

Parameters
----------
name : str
    Name of the sequence variable in the output dataset.
out : PathType
    Path to the output Zarr store where the data will be saved.
    Usually something like `/path/to/dataset_name.zarr`.
fasta : PathType
    Path to the input FASTA file.
batch_size : int
    Number of sequences to read at a time. Use as many as you can fit in memory.
fixed_length : bool
    Whether your sequences have a fixed length or not. If they do, the data will 

In [5]:
sdata = read_flat_fasta(
    fasta=fname,
    name="seq",
    out=fname.with_suffix(".zarr"),
    fixed_length=False,
    batch_size=1000
)
sdata

100%|██████████| 7/7 [00:00<00:00, 1634.11it/s]

<xarray.Dataset>
Dimensions:    (_sequence: 7)
Coordinates:
  * _sequence  (_sequence) object 'chr1' 'chr2' 'chr3' ... 'chr5' 'chr6' 'chr7'
Data variables:
    seq        (_sequence) object dask.array<chunksize=(7,), meta=np.ndarray>

In [7]:
sdata.seq[0].values

array(b'CGACTACTACCGACTAACTGACTGATGATGATGCATGCTGATGCTGAACTGACTAGCACTGCATGACTGATGACTGACTGTACTCCTACCATGACTATCCTAGTGCTGACCTGACTGATGCTGACTGACTGCATATGCACTGACTGACTCTACATGACTGACTCACTCATCTGACATATCCATGCTGCATACTCATGATCATGCATGCATCATACTCATGCATGACTGACTCATGATGCACATACTACTGCAGTCTGCATCATGCATGCATGCATGCACATCAT',
      dtype='|S284')

Under the hood, each of these `read_` functions is actually a specific call to a more general composable function.

## `FlatReader`
As the name suggests, these readers specialize in reading data from what we call flat files. Flat files are files that contain data that is not inherently associated with a genomic region. Examples of reading from flat files include reading from tabular data (`Table` reader) and a fasta where each entry correpsonds to an example used for modeling (`FlatFasta` reader).

In [8]:
from seqdata import FlatFASTA
from seqdata import from_flat_files

In [9]:
FlatFASTA?

Init signature:
FlatFASTA(
    name: str,
    fasta: Union[str, pathlib.Path],
    batch_size: int,
    n_threads: int = 1,
) -> None
Docstring:     
Helper class that provides a standard way to create an ABC using
inheritance.
Init docstring:
Reader for flat FASTA files.

Parameters
----------
name : str
    Name of the sequence array in resulting SeqData Zarr.
fasta : str, Path
    Path to FASTA file.
batch_size : int
    Number of sequences to read at once.
n_threads : int, default 1
    Number of threads to use for reading the FASTA file.

Returns
-------
None
File:           ~/projects/ML4GLand/SeqData/seqdata/_io/readers/fasta.py
Type:           ABCMeta
Subclasses:     

In [11]:
sdata = from_flat_files(
        FlatFASTA(fasta=fname, name="seq",batch_size=1000),
        path=fname.with_suffix(".zarr"),
        fixed_length=False,
        overwrite=True
    )
sdata

100%|██████████| 7/7 [00:00<00:00, 1533.49it/s]


<xarray.Dataset>
Dimensions:    (_sequence: 7)
Coordinates:
  * _sequence  (_sequence) object 'chr1' 'chr2' 'chr3' ... 'chr5' 'chr6' 'chr7'
Data variables:
    seq        (_sequence) object dask.array<chunksize=(7,), meta=np.ndarray>

If we wanted to read two

In [16]:
fname.with_suffix(".tangermeme2.fa")

PosixPath('/cellar/users/aklie/projects/ML4GLand/SeqData/tests/data/tangermeme.tangermeme2.fa')

In [20]:
fname.with_name?

Signature: fname.with_name(name)
Docstring: Return a new path with the file name changed.
File:      ~/opt/miniconda3/envs/eugene_tools/lib/python3.11/pathlib.py
Type:      method

In [22]:
sdata = from_flat_files(
        FlatFASTA(fasta=fname, name="seq",batch_size=1000),
        FlatFASTA(fasta=fname, name="seq2",batch_size=1000),
        path=fname.with_suffix(".zarr"),
        fixed_length=False,
        overwrite=True
    )
sdata

100%|██████████| 7/7 [00:00<00:00, 2638.16it/s]


<xarray.Dataset>
Dimensions:    (_sequence: 7)
Coordinates:
  * _sequence  (_sequence) object 'chr1' 'chr2' 'chr3' ... 'chr5' 'chr6' 'chr7'
Data variables:
    seq        (_sequence) object dask.array<chunksize=(7,), meta=np.ndarray>
    seq2       (_sequence) object dask.array<chunksize=(7,), meta=np.ndarray>

In [ ]:
sdata = from_flat_files(
        FlatFASTA(fasta=fname, name="seq",batch_size=1000),
        FlatFASTA(fasta=fname.with_suffix("tangerme"), name="seq2",batch_size=1000),
        path=fname.with_suffix(".zarr"),
        fixed_length=False,
        overwrite=True
    )
sdata

## `RegionReader`
Region readers require a genome

In [2]:
from seqdata import read_bigwig

In [3]:
# Get file name
bw_fname = Path(sd.__file__).resolve().parent.parent / 'tests' / 'data' / 'tangermeme.bw'
bed_fname = Path(sd.__file__).resolve().parent.parent / 'tests' / 'data' / 'tangermeme.bed'
fasta_fname = Path(sd.__file__).resolve().parent.parent / 'tests' / 'data' / 'tangermeme.fa'
bw_fname, bed_fname, fasta_fname

(PosixPath('/cellar/users/aklie/projects/ML4GLand/SeqData/tests/data/tangermeme.bw'),
 PosixPath('/cellar/users/aklie/projects/ML4GLand/SeqData/tests/data/tangermeme.bed'),
 PosixPath('/cellar/users/aklie/projects/ML4GLand/SeqData/tests/data/tangermeme.fa'))

In [4]:
# Get file name
bw_fnames = [bw_fname, Path(sd.__file__).resolve().parent.parent / 'tests' / 'data' / 'tangermeme2.bw']
bw_fnames

[PosixPath('/cellar/users/aklie/projects/ML4GLand/SeqData/tests/data/tangermeme.bw'),
 PosixPath('/cellar/users/aklie/projects/ML4GLand/SeqData/tests/data/tangermeme2.bw')]

In [5]:
sdata2 = read_bigwig(
    bigwigs=bw_fnames,  # bigwig files
    fasta=fasta_fname,  # reference genome
    seq_name="seq",  # name of resulting xarray variable containing sequences
    cov_name="cov",  # name of resulting xarray variable containing coverage
    bed=bed_fname,  # bed file with regions to extract
    samples=["tangermeme", "tangermeme2"],  # sample names
    out=bw_fname.with_suffix(".zarr"),
    fixed_length=True,  # whether all sequences are the same length
    batch_size=1000,  # number of sequences to load at once
    n_jobs=1,  # number of parallel jobs
    overwrite=True,  # overwrite the output file if it exists
)
sdata2

/cellar/users/aklie/opt/miniconda3/envs/eugene_tools/lib/python3.11/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
100%|██████████| 5/5 [00:00<00:00, 17939.71it/s]


<xarray.Dataset>
Dimensions:     (_sequence: 5, cov_sample: 2, _length: 20)
Coordinates:
  * cov_sample  (cov_sample) object 'tangermeme' 'tangermeme2'
Dimensions without coordinates: _sequence, _length
Data variables:
    chrom       (_sequence) object dask.array<chunksize=(5,), meta=np.ndarray>
    chromEnd    (_sequence) int64 dask.array<chunksize=(5,), meta=np.ndarray>
    chromStart  (_sequence) int64 dask.array<chunksize=(5,), meta=np.ndarray>
    cov         (_sequence, cov_sample, _length) float32 dask.array<chunksize=(5, 1, 20), meta=np.ndarray>
    seq         (_sequence, _length) |S1 dask.array<chunksize=(5, 20), meta=np.ndarray>
    strand      (_sequence) object dask.array<chunksize=(5,), meta=np.ndarray>
Attributes:
    length_dim:    _length
    max_jitter:    0
    sequence_dim:  _sequence

Similary to with Flat files, there is a composable version of reading from regions

In [5]:
from seqdata import from_region_files

In [6]:
# Build SeqData
sdata = sd.from_region_files(
    sd.GenomeFASTA('seq',
        fasta_fname,
        batch_size=1000,
    ),
    sd.BigWig(
        'cov',
        [str(bw_fname)],
        ["tangermeme"],
        batch_size=1000,
    ),
    path=bw_fname.with_suffix(".zarr"),
    fixed_length=50,
    bed=bed_fname,
    overwrite=True,
)
sdata

/cellar/users/aklie/opt/miniconda3/envs/eugene_tools/lib/python3.11/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
100%|██████████| 5/5 [00:00<00:00, 11657.32it/s]


<xarray.Dataset>
Dimensions:     (_sequence: 5, cov_sample: 1, _length: 50)
Coordinates:
  * cov_sample  (cov_sample) object 'tangermeme'
Dimensions without coordinates: _sequence, _length
Data variables:
    chrom       (_sequence) object dask.array<chunksize=(5,), meta=np.ndarray>
    chromEnd    (_sequence) int64 dask.array<chunksize=(5,), meta=np.ndarray>
    chromStart  (_sequence) int64 dask.array<chunksize=(5,), meta=np.ndarray>
    cov         (_sequence, cov_sample, _length) float32 dask.array<chunksize=(5, 1, 50), meta=np.ndarray>
    seq         (_sequence, _length) |S1 dask.array<chunksize=(5, 50), meta=np.ndarray>
    strand      (_sequence) object dask.array<chunksize=(5,), meta=np.ndarray>
Attributes:
    length_dim:    _length
    max_jitter:    0
    sequence_dim:  _sequence

In [8]:
# Build SeqData
sdata = sd.from_region_files(
    sd.GenomeFASTA('seq',
        fasta_fname,
        batch_size=1000,
    ),
    sd.BigWig(
        'cov',
        [str(bw_fname)],
        ["tangermeme"],
        batch_size=1000,
    ),
    sd.BigWig(
        'cov2',
        [str(bw_fname)],
        ["tangermeme2"],
        batch_size=1000,
    ),
    path=bw_fname.with_suffix(".zarr"),
    fixed_length=50,
    bed=bed_fname,
    overwrite=True,
)
sdata

100%|██████████| 5/5 [00:00<00:00, 24442.33it/s]


<xarray.Dataset>
Dimensions:      (_sequence: 5, cov_sample: 1, _length: 50, cov2_sample: 1)
Coordinates:
  * cov2_sample  (cov2_sample) object 'tangermeme2'
  * cov_sample   (cov_sample) object 'tangermeme'
Dimensions without coordinates: _sequence, _length
Data variables:
    chrom        (_sequence) object dask.array<chunksize=(5,), meta=np.ndarray>
    chromEnd     (_sequence) int64 dask.array<chunksize=(5,), meta=np.ndarray>
    chromStart   (_sequence) int64 dask.array<chunksize=(5,), meta=np.ndarray>
    cov          (_sequence, cov_sample, _length) float32 dask.array<chunksize=(5, 1, 50), meta=np.ndarray>
    cov2         (_sequence, cov2_sample, _length) float32 dask.array<chunksize=(5, 1, 50), meta=np.ndarray>
    seq          (_sequence, _length) |S1 dask.array<chunksize=(5, 50), meta=np.ndarray>
    strand       (_sequence) object dask.array<chunksize=(5,), meta=np.ndarray>
Attributes:
    length_dim:    _length
    max_jitter:    0
    sequence_dim:  _sequence